# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [2]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [3]:
document = ET.parse( './data/mondial_database.xml' )

In [17]:
#10 countries with the lowest infant mortality rate
import pandas as pd
col1 = []
col2 = []
for child in document.getroot():
    if (child.find('infant_mortality') != None) & (child.find('name') != None) :
        col1.append(child.find('infant_mortality').text)
        col2.append(child.find('name').text)
d = {'country' : col2,
     'mortality' : col1}
df = pd.DataFrame(d)
df.mortality = df.mortality.astype('float')
df.sort('mortality').head(10)

,country,mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


In [25]:
#10 cities with the largest population
col1 = []
col2 = []
for element in document.iterfind('country'):
    for subelement in element.getiterator('city'):
        if(subelement.find('population') != None):
            col1.append(subelement.find('name').text)
            col2.append(subelement.findall('population')[-1].text)
    
d = {'population' : col2,
     'city' : col1}
df = pd.DataFrame(d)
df['population'] = df['population'].astype(float)
df = df.sort('population', ascending = False)
df.head(10)

,city,population
1251,Shanghai,22315474
707,Istanbul,13710512
1421,Mumbai,12442373
443,Moskva,11979529
1250,Beijing,11716620
2594,São Paulo,11152344
1252,Tianjin,11090314
974,Guangzhou,11071424
1467,Delhi,11034555
977,Shenzhen,10358381


In [34]:
#10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
col1 = []
col2 = []
col3 = []
for child in document.getroot():
    if (child.find('ethnicgroup') != None):
        for i in range(len(child.findall('ethnicgroup'))):
            col1.append(child.findall('ethnicgroup')[i].text)
            col2.append(child.findall('ethnicgroup')[i].attrib['percentage'])
            col3.append(child.findall('population')[-1].text)

d = {'ethnic group' : col1,
     'percentage' : col2,
     'population' : col3 }

df = pd.DataFrame(d)
df['population'] = df['population'].astype(float)
df['percentage'] = df['percentage'].astype(float)
df['ethnic population'] = df['population'] * df['percentage'] /100
df.groupby('ethnic group')['ethnic population'].sum().order('real population', ascending = False).head(10)

ethnic group
Han Chinese     1.245059e+09
Indo-Aryan      8.718156e+08
European        4.948722e+08
African         3.183251e+08
Dravidian       3.027137e+08
Mestizo         1.577344e+08
Bengali         1.467769e+08
Russian         1.318570e+08
Japanese        1.265342e+08
Malay           1.219936e+08
Name: ethnic population, dtype: float64

In [40]:
#name and country of a) longest river, b) largest lake and c) airport at highest elevation

#Longest River
col1 = []
col2 = []
col3 = []

#& (child.find('country') != None)
for child in document.getroot():
    if ((child.find('length') != None) & ((child.find('located') != None))):
        col1.append(child.find('length').text)
        col2.append(child.find('located').attrib['country']) 
        col3.append(child.find('name').text)
    
d = {'length' : col1,
     'country' : col2,
     'river name' : col3 }

df = pd.DataFrame(d)
df['length'] = df['length'].astype(float)
df = df.sort('length')
df.tail(1)

,country,length,river name
164,CO,6448,Amazonas


In [45]:
#b) largest lake 
col1 = []
col2 = []
col3 = []

for child in document.iterfind('lake'):
    if ((child.find('area') != None) & ((child.find('located') != None))):
        col1.append(child.find('area').text) 
        col2.append(child.find('located').attrib['country'])
        col3.append(child.find('name').text)

d = {'area' : col1,
     'country' : col2,
     'Lake name' : col3 }

df = pd.DataFrame(d)
df['area'] = df['area'].astype(float)
df = df.sort('area', ascending = False)
df.head(1)


,Lake name,area,country
52,Caspian Sea,386400,R


In [47]:
#c) airport at highest elevation
col1 = []
col2 = []
col3 = []

#& (child.find('country') != None)
for child in document.iterfind('airport'):
    if ((child.find('elevation') != None) & (child.attrib['country'] != None)):
        col1.append(child.find('elevation').text)
        col2.append(child.attrib['country'])
        col3.append(child.find('name').text)

d = {'elevation' : col1,
     'country' : col2,
     'airport' : col3 }

df = pd.DataFrame(d)
df['elevation'] = df['elevation'].astype(float)
df = df.sort('elevation', ascending = False)
df.head(1)


,airport,country,elevation
80,El Alto Intl,BOL,4063
